<a href="https://colab.research.google.com/github/ovbystrova/dpl/blob/master/notebooks/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from collections import Counter
import pickle


import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk import tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# TODO Прогнать когда будет 250к каждого типа, записать статистику. 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_csv('/content/drive/My Drive/dpl_dataset.csv')
fakes = df[df['label']=='fake']
reals = df[df['label']=='real']

reals.reset_index(drop=True, inplace=True)

In [7]:
df['sampling'].value_counts()

No sampling    440000
temperature    120000
argmax         110000
top_k          110000
nucleus        100000
Name: sampling, dtype: int64

# Sentence Length 

In [8]:
# drop all that is less than 2 sentences
ids_fdrop = [ind  for ind, el in enumerate(list(fakes['text'])) if len(tokenize.sent_tokenize(el))<2]
ids_rdrop = [ind  for ind, el in  enumerate(list(reals['text'])) if len(tokenize.sent_tokenize(el))<2]
ids_drop = list(set(ids_fdrop + ids_rdrop))
len(ids_drop)

9031

In [9]:
len(ids_fdrop), len(ids_rdrop)

(9027, 5)

In [0]:
fakes.drop(np.array(ids_drop), inplace=True)
reals.drop(np.array(ids_drop), inplace=True)
fakes.reset_index(drop=True, inplace=True)
reals.reset_index(drop=True, inplace=True)

In [11]:
fakes.shape, reals.shape

((430969, 3), (430969, 3))

In [12]:
fakes.head(), reals.head()

(                                                text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  fake  temperature
 1  WASHINGTON, Dec. 23 - The National Security Ag...  fake  temperature
 2  IF outsized executive pay has indeed become a ...  fake  temperature
 3  BY A.J. Miller, Jr.  The three men will make t...  fake  temperature
 4  Spinach has terrorized generations of veggie-p...  fake  temperature,
                                                 text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  real  No sampling
 1  WASHINGTON, Dec. 23 - The National Security Ag...  real  No sampling
 2  IF outsized executive pay has indeed become a ...  real  No sampling
 3  BY A.J. BENZA & MICHAEL LEWITTES If Simon Rex ...  real  No sampling
 4  Spinach has terrorized generations of veggie-p...  real  No sampling)

In [13]:
fakes.tail(), reals.tail()

(                                                     text label     sampling
 430964  A Brooklyn teacher who was arrested in 2011 on...  fake  temperature
 430965  A baby-faced Bronx teenager was executed on th...  fake  temperature
 430966  Published: Friday, May 22, 2015, 7:07 AM Updat...  fake  temperature
 430967  Authorities believe more than one person is re...  fake  temperature
 430968  The Yankees are going down, and no part of the...  fake  temperature,
                                                      text label     sampling
 430964  A Brooklyn teacher who was arrested in 2011 on...  real  No sampling
 430965  A baby-faced Bronx teenager was executed on th...  real  No sampling
 430966  Published: Friday, May 22, 2015, 7:07 AM Updat...  real  No sampling
 430967  Authorities believe more than one person is re...  real  No sampling
 430968  The Yankees are going down, and no part of the...  real  No sampling)

In [0]:
# save only first two sentences

fakes['text'] = fakes['text'].apply(lambda x: ' '.join(tokenize.sent_tokenize(x)[:2]))
reals['text'] = reals['text'].apply(lambda x: ' '.join(tokenize.sent_tokenize(x)[:2]))

In [16]:
ids_fdrop = [ind  for ind, el in enumerate(list(fakes['text'])) if len(tokenize.sent_tokenize(el))>2]
ids_rdrop = [ind  for ind, el in  enumerate(list(reals['text'])) if len(tokenize.sent_tokenize(el))>2]
ids_drop = list(set(ids_fdrop + ids_rdrop))
len(ids_drop)

1600

In [0]:
fakes.drop(np.array(ids_drop), inplace=True)
reals.drop(np.array(ids_drop), inplace=True)
fakes.reset_index(drop=True, inplace=True)
reals.reset_index(drop=True, inplace=True)

In [18]:
fakes.head(), reals.head()

(                                                text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  fake  temperature
 1  WASHINGTON, Dec. 23 - The National Security Ag...  fake  temperature
 2  IF outsized executive pay has indeed become a ...  fake  temperature
 3                                BY A.J. Miller, Jr.  fake  temperature
 4  Spinach has terrorized generations of veggie-p...  fake  temperature,
                                                 text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  real  No sampling
 1  WASHINGTON, Dec. 23 - The National Security Ag...  real  No sampling
 2  IF outsized executive pay has indeed become a ...  real  No sampling
 3  BY A.J. BENZA & MICHAEL LEWITTES If Simon Rex ...  real  No sampling
 4  Spinach has terrorized generations of veggie-p...  real  No sampling)

In [19]:
fakes.tail(), reals.tail()

(                                                     text label     sampling
 429364  A Brooklyn teacher who was arrested in 2011 on...  fake  temperature
 429365  A baby-faced Bronx teenager was executed on th...  fake  temperature
 429366  Published: Friday, May 22, 2015, 7:07 AM Updat...  fake  temperature
 429367  Authorities believe more than one person is re...  fake  temperature
 429368  The Yankees are going down, and no part of the...  fake  temperature,
                                                      text label     sampling
 429364  A Brooklyn teacher who was arrested in 2011 on...  real  No sampling
 429365  A baby-faced Bronx teenager was executed on th...  real  No sampling
 429366  Published: Friday, May 22, 2015, 7:07 AM Updat...  real  No sampling
 429367  Authorities believe more than one person is re...  real  No sampling
 429368  The Yankees are going down, and no part of the...  real  No sampling)

In [20]:
f = 0
for el in list(fakes['text']):
    if len(tokenize.sent_tokenize(el)) > 2:
        f +=1
r =0
for el in list(reals['text']):
    if len(tokenize.sent_tokenize(el)) > 2:
        r +=1
f, r

(0, 0)

# Drop duplicates

In [21]:
fakes.describe(), reals.describe()

(                                                     text   label     sampling
 count                                              429369  429369       429369
 unique                                             427727       1            4
 top     This post has been corrected. The following is...    fake  temperature
 freq                                                   98  429369       118001,
                                                      text   label     sampling
 count                                              429369  429369       429369
 unique                                             408063       1            1
 top     This post is in partnership with Time. The art...    real  No sampling
 freq                                                  139  429369       429369)

In [22]:
fakes.shape, reals.shape

((429369, 3), (429369, 3))

In [0]:
fakes_clean = fakes.drop_duplicates(inplace=False)
reals_clean = reals.drop_duplicates(inplace=False)

In [24]:
fakes_clean.shape, fakes.shape, reals_clean.shape, reals.shape

((427855, 3), (429369, 3), (408063, 3), (429369, 3))

In [25]:
ids_fdrop = [ind for ind in fakes.index if ind not in fakes_clean.index]
ids_rdrop = [ind for ind in reals.index if ind not in reals_clean.index]
ids_drop = list(set(ids_fdrop + ids_rdrop))
len(ids_drop)

22117

In [0]:
fakes.drop(np.array(ids_drop), inplace=True)
reals.drop(np.array(ids_drop), inplace=True)
fakes.reset_index(drop=True, inplace=True)
reals.reset_index(drop=True, inplace=True)

In [27]:
fakes.head(), reals.head()

(                                                text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  fake  temperature
 1  WASHINGTON, Dec. 23 - The National Security Ag...  fake  temperature
 2  IF outsized executive pay has indeed become a ...  fake  temperature
 3                                BY A.J. Miller, Jr.  fake  temperature
 4  Spinach has terrorized generations of veggie-p...  fake  temperature,
                                                 text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  real  No sampling
 1  WASHINGTON, Dec. 23 - The National Security Ag...  real  No sampling
 2  IF outsized executive pay has indeed become a ...  real  No sampling
 3  BY A.J. BENZA & MICHAEL LEWITTES If Simon Rex ...  real  No sampling
 4  Spinach has terrorized generations of veggie-p...  real  No sampling)

In [28]:
fakes.tail(), reals.tail()

(                                                     text label     sampling
 407247  A Brooklyn teacher who was arrested in 2011 on...  fake  temperature
 407248  A baby-faced Bronx teenager was executed on th...  fake  temperature
 407249  Published: Friday, May 22, 2015, 7:07 AM Updat...  fake  temperature
 407250  Authorities believe more than one person is re...  fake  temperature
 407251  The Yankees are going down, and no part of the...  fake  temperature,
                                                      text label     sampling
 407247  A Brooklyn teacher who was arrested in 2011 on...  real  No sampling
 407248  A baby-faced Bronx teenager was executed on th...  real  No sampling
 407249  Published: Friday, May 22, 2015, 7:07 AM Updat...  real  No sampling
 407250  Authorities believe more than one person is re...  real  No sampling
 407251  The Yankees are going down, and no part of the...  real  No sampling)

In [29]:
fakes.describe()

,text,label,sampling
count,407252,407252,407252
unique,407247,1,4
top,Former President George H.W. Bush.,fake,temperature
freq,2,407252,108273


In [30]:
reals.describe()

,text,label,sampling
count,407252,407252,407252
unique,407252,1,1
top,Drugs shouldn't be approved by the Food and Dr...,real,No sampling
freq,1,407252,407252


# Counter

In [0]:
fakes_tokens = fakes['text'].apply(lambda x: x.split(' '))
reals_tokens = reals['text'].apply(lambda x: x.split(' '))

fake_split = Counter(x for xs in fakes_tokens for x in set(xs))
real_split = Counter(x for xs in reals_tokens for x in set(xs))

In [0]:
fake_split.most_common(20), real_split.most_common(20)

([('the', 306126),
  ('a', 255427),
  ('of', 237154),
  ('to', 233236),
  ('in', 215643),
  ('and', 204753),
  ('The', 147617),
  ('on', 130350),
  ('for', 129045),
  ('is', 127671),
  ('that', 120385),
  ('with', 92565),
  ('at', 88895),
  ('was', 87584),
  ('has', 82455),
  ('by', 70808),
  ('from', 69187),
  ('as', 68488),
  ('an', 63704),
  ('be', 59258)],
 [('the', 317340),
  ('a', 255371),
  ('of', 248451),
  ('to', 238233),
  ('and', 224050),
  ('in', 223469),
  ('on', 137224),
  ('for', 129814),
  ('is', 116204),
  ('that', 114687),
  ('The', 111141),
  ('with', 94572),
  ('at', 88548),
  ('has', 76575),
  ('was', 76050),
  ('from', 72375),
  ('as', 70519),
  ('by', 67636),
  ('an', 65433),
  ('his', 59783)])

# Save samplings datasets

In [32]:
samplings = fakes['sampling'].unique()

for sampling in samplings:
    
    fake_sampl = fakes[fakes['sampling']==sampling]
    real_sampl = reals.iloc[fake_sampl.index]
    assert real_sampl.shape == fake_sampl.shape, 'Different lenths of real and fake samplings'

    df_sampl = pd.concat([fake_sampl, real_sampl], axis=0)
    df_sampl = shuffle(df_sampl)
    df_sampl.reset_index(drop=True, inplace=True)
    
    df_sampl.to_csv('/content/drive/My Drive/data/sampling_same_{}.csv'.format(sampling), index=False)
    print('{} texts for {} sampling type were saved in sampling_same_{}.csv'.format(df_sampl.shape[0], sampling, sampling))

216546 texts for temperature sampling type were saved in sampling_same_temperature.csv
204466 texts for top_k sampling type were saved in sampling_same_top_k.csv
186806 texts for nucleus sampling type were saved in sampling_same_nucleus.csv
206686 texts for argmax sampling type were saved in sampling_same_argmax.csv


# Shuffle and save

In [0]:
df = pd.concat([fakes, reals], axis=0)
df.reset_index(drop=True, inplace=True)

In [0]:
df.head()

,text,label,sampling
0,"HAMBURG, Germany, June 3  As he left the socc...",fake,temperature
1,"WASHINGTON, Dec. 23 - The National Security Ag...",fake,temperature
2,IF outsized executive pay has indeed become a ...,fake,temperature
3,"BY A.J. Miller, Jr.",fake,temperature
4,Spinach has terrorized generations of veggie-p...,fake,temperature


In [0]:
df.tail()

,text,label,sampling
745693,"This article is part of BrandSpeak, a program ...",real,No sampling
745694,Facebook stock shot up more than 4% in early t...,real,No sampling
745695,"At its Windows Phone 8 event in New York City,...",real,No sampling
745696,Nokia unveiled on Wednesday in New York City t...,real,No sampling
745697,Grammy-nominated rapper B.o.B is an alien — at...,real,No sampling


In [0]:
df = shuffle(df)
df.reset_index(drop=True, inplace=True)

In [0]:
df.head()

,text,label,sampling
0,agementA Sotheby's employee poses next to a wo...,real,No sampling
1,Your Post Has Been Launched! Is raising a chil...,real,No sampling
2,NEW YORK (CNNMoney.com) -- Sometimes the winne...,fake,temperature
3,A Minnesota family has been in a state of limb...,real,No sampling
4,(CNN) -- Saudi Arabia named Prince Salman bin ...,real,No sampling


In [0]:
df.tail()

,text,label,sampling
745693,"Eat, Pray, Love author Elizabeth Gilbert is us...",fake,nucleus
745694,CHICAGO – Four people have been charged in inv...,real,No sampling
745695,It took Timothy Hussey nearly a year to reboun...,fake,argmax
745696,Don Draper in Mad Men. Gordon Gekko in Wall St...,real,No sampling
745697,"Updated OCT 29, 2014 1:56p ET A Premier League...",real,No sampling


In [0]:
fake = list(fakes['text'])
real = list(reals['text'])

df.to_csv('/content/drive/My Drive/data/postpoc_dpl_dataset.csv', index=False)

In [0]:
with open('/content/drive/My Drive/data/fake', 'wb') as fp:
    pickle.dump(fake, fp)

with open('/content/drive/My Drive/data/real', 'wb') as fp:
    pickle.dump(real, fp)

In [0]:
# with open ('fake', 'rb') as fp:
#     itemlist = pickle.load(fp)